# Introduction

This notebook will walk you through how to accelerate model training with Composer. We'll start by training a baseline ResNet56 on CIFAR10, then see how training efficiency improves as we add speed-up methods.


# Install Composer

We'll start by installing composer

In [ ]:
!pip install git+https://github.com/mosaicml/composer.git@dev

# Set Up Our Workspace

In this section we'll set up our workspace. We'll import the necessary packages, and setup our dataset and trainer. First, the imports:

In [ ]:
import time

import torch

import composer
from torchvision import datasets, transforms

torch.manual_seed(42) # For replicability

Next, we instantiate our CIFAR10 dataset and dataloader. Composer has it's own CIFAR10 dataset and dataloaders, but this walkthrough focuses on how to use Composer's algorithms, so we'll stick with the Torchvision CIFAR10 and PyTorch dataloader for the sake of familiarity.

In [ ]:
data_directory = "../data"

# Normalization constants
mean = (0.507, 0.487, 0.441)
std = (0.267, 0.256, 0.276)

batch_size = 1024

cifar10_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

train_dataset = datasets.CIFAR10(data_directory, train=True, download=True, transform=cifar10_transforms)
test_dataset = datasets.CIFAR10(data_directory, train=False, download=True, transform=cifar10_transforms)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


Next, we create our model. We're using composer's built-in ResNet56. To use your own custom model, please see the [custom models tutorial](https://docs.mosaicml.com/en/v0.3.1/tutorials/adding_models_datasets.html#models).

In [ ]:
from composer import models
model = models.CIFAR10_ResNet56()

The trainer will handle instantiating the optimizer, but first we need to create the optimizer and LR scheduler. We're using [MosaicML's SGD with decoupled weight decay](https://arxiv.org/abs/1711.05101):

In [ ]:
optimizer = composer.optim.DecoupledSGDW(
    model.parameters(), # Model parameters to update
    lr=0.05, # Peak learning rate
    momentum=0.9,
    weight_decay=2.0e-3 # If this looks large, it's because its not scaled by the LR as in non-decoupled weight decay
    )

We'll assume this is being run on Colab, which means training for hundreds of epochs would take a very long time. Instead we'll train our baseline model for three epochs. The first epoch will be linear warmup. We achieve this by instantiating a `WarmUpLRHparams` class.

In [ ]:
## Having to manually compute the number of iterations is ungainly

warmup = composer.optim.WarmUpLR(
    optimizer, # Optimizer
    warmup_iters=25, # Number of iterations to warmup over. 50k samples * 1 batch/2048 samples
    warmup_method="linear", # Linear warmup
    warmup_factor=1e-4, # Initial LR = LR * warmup_factor
    interval="step", # Update LR with stepwise granularity for superior results
)

We'll also use cosine decay for the next two epochs:

In [ ]:
## Seems like there's no way to have stepwise resolution for cosine decay without using CosineAnnealingLRHparams :(

# decay = torch.optim.lr_scheduler.CosineAnnealingLR(
#     optimizer,
#     T_max=50 # 2 epochs == 50 steps
# )

An LR schedule is provided to the trainer as a list of SchedulerHparams objects. In this case we just have one, but creating a more complex LR schedule is as simple as collecting the SchedulerHparams objects in a list.

In [ ]:
lr_schedule = [warmup] # Complex LR schedules achieved by [warmup_scheduler_object, decay_scheduler_object, ...]

# Train a Baseline Model

And now we create our trainer:

In [ ]:
## Need in-memory logging to store and plot results

train_epochs = "3ep" # Train for 3 epochs because we're assuming Colab environment and hardware
device = "gpu" # Train on the GPU

trainer = composer.trainer.Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    max_duration=train_epochs,
    optimizers=optimizer,
    schedulers=lr_schedule,
    device=device
)



# Would be nice to be able to plot something here

We train and measure the training time below.

In [ ]:
start_time = time.perf_counter()
trainer.fit()
end_time = time.perf_counter()
print(f"It took {end_time - start_time:0.4f} seconds to train")

If you're running this on Colab, the runtime will vary a lot based on the instance. We found that the three epochs of training could take anywhere from 120-550 seconds to run, and the mean validation accuracy was typically in the range of 25%-40%.

# Use Algorithms to Speed Up Training

One of the things we're most excited about at MosaicML is our speed-up algorithms. We used these algorithms to [speed up training of ResNet50 on ImageNet by up to 3.4x](https://app.mosaicml.com/explorer/imagenet). Let's try applying a few algorithms to make our ResNet56 more efficient.

We'll start with [ColOut](https://docs.mosaicml.com/en/v0.3.1/method_cards/col_out.html), which is an in-house invention. Colout drops rows and columns of an image with probability *p*. It's a little bit like [Random Erasing](https://arxiv.org/abs/1708.04896) except it reduces the size of the image, which can increase data throughput and speed up training.

In [ ]:
colout = composer.algorithms.ColOut()

Let's also use [BlurPool](https://docs.mosaicml.com/en/v0.3.1/method_cards/blurpool.html), which increases accuracy by applying a spatial low-pass filter before the pool in max pooling and whenever using a strided convolution.

In [ ]:
blurpool = composer.algorithms.BlurPool(
    replace_convs=True, # Blur before convs
    replace_maxpools=True, # Blur before max-pools
    blur_first=True # Blur before conv/max-pool
)

Our final algorithm in our improved training recipe is [Progressive Image Resizing](https://docs.mosaicml.com/en/v0.3.1/method_cards/progressive_resizing_vision.html). Progressive Image Resizing initially shrinks the size of training images and slowly scales them back to their full size over the course of training. It increases throughput during the early phase of training, when the network may learn coarse-grained features that do not require details lost by reducing image resolution.

In [ ]:
prog_resize = composer.algorithms.ProgressiveResizing(
    initial_scale=.6, # Size of images at the beginning of training = .6 * default image size
    finetune_fraction=0.33 # Train on default size images for 0.5 of total training time.
)

We'll assemble all our algorithms into a list to pass to our trainer.

In [ ]:
algorithms = [colout, blurpool, prog_resize]

Now let's instantiate our model, optimizer, scheduler, and trainer again.

In [ ]:
model = models.CIFAR10_ResNet56()

optimizer = composer.optim.DecoupledSGDW(
    model.parameters(),
    lr=0.05,
    momentum=0.9,
    weight_decay=2.0e-3
    )

warmup = composer.optim.WarmUpLR(
    optimizer, # Optimizer
    warmup_iters=25, # Number of iterations to warmup over. 50k samples * 1 batch/2048 samples
    warmup_method="linear", # Linear warmup
    warmup_factor=1e-4, # Initial LR = LR * warmup_factor
    interval="step", # Update LR with stepwise granularity for superior results
)

lr_schedule = [warmup]

trainer = composer.trainer.Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    max_duration=train_epochs,
    optimizers=optimizer,
    schedulers=lr_schedule,
    device=device,
    algorithms=algorithms # Adding algorithms this time
)

And let's get training!

In [ ]:
start_time = time.perf_counter()
trainer.fit()
end_time = time.perf_counter()
print(f"It took {end_time - start_time:0.4f} seconds to train")

Again, the runtime will vary based on the instance, but we found that it took about a 0.43x-0.75x as long to train (a 1.3x-2.3x speedup, which corresponds to 90-400 seconds) relative to the baseline recipe without augmentations. We also found that validation accuracy was similar for the algorithm-enhanced and baseline recipes.

Because ColOut and Progressive Resizing increase data throughput (i.e. more samples per sceond), we can train for more iterations in the same amount of wall clock time. Let's train for another epoch!

In [ ]:
train_epochs = "1ep"

We'll need to set up our optimizer, scheduler, and trainer again:

In [ ]:
optimizer = composer.optim.DecoupledSGDW(
    model.parameters(),
    lr=0.05,
    momentum=0.9,
    weight_decay=2.0e-3
    )

# The Composer trainer defaults to cosine decay if no schedule is passed to it,
# so let's creata constant LR schedule
flat_schedule = composer.optim.ConstantLR(
    optimizer
)

lr_schedule = [flat_schedule]

trainer = composer.trainer.Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    max_duration='5ba',
    optimizers=optimizer,
    schedulers=lr_schedule,
    device=device,
    algorithms=[colout, blurpool] # Adding algorithms this time
)

start_time = time.perf_counter()
trainer.fit()
end_time = time.perf_counter()
print(f"It took {end_time - start_time:0.4f} seconds to train")

We found that using these speed-up algorithms for four epochs resulted in runtime similar to or less than three epochs *without* speed-up algorithms (120-550 seconds, depending on the instance), and that they usually improved validation accuracy by 5-15 percentage points, yielding validation accuracy in the range of 30%-50%.

# You did it! Now come get involved with MosaicML!

Hopefully you're now comfortable with the basics of training with Composer. We'd love for you to get involved with MosaicML community in any of these ways:

## [Star Composer on GitHub](https://github.com/mosaicml/composer)

Stay up-to-date and help make others aware of our work by [starring Composer on GitHub](https://github.com/mosaicml/composer).

## [Join the MosaicML Slack](https://join.slack.com/t/mosaicml-community/shared_invite/zt-w0tiddn9-WGTlRpfjcO9J5jyrMub1dg)

Head on over to the [MosaicML slack](https://join.slack.com/t/mosaicml-community/shared_invite/zt-w0tiddn9-WGTlRpfjcO9J5jyrMub1dg) to join other ML efficiency enthusiasts. Come for the paper discussions, stay for the memes!

## Contribute to Composer

Is there a bug you noticed or a feature you'd like? File an [issue](https://github.com/mosaicml/composer/issues) or make [pull request](https://github.com/mosaicml/composer/pulls)!